In [ ]:
import pandas as pd
import numpy as np
#import bootstrapped.bootstrap as bts
#import bootstrapped.stats_functions as bs_stats
import os
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pylab as pl
from os import path
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams['font.family'] = "sans-serif"

In [ ]:
base_folder_string = 'output_0901_tiling600_ConstrainingLEFsize_run2'
folders = [foldername for foldername in os.listdir('.') if foldername.startswith(base_folder_string)]

base_stochasticity = 1 # use to convert step to time



for m in range(len(folders)):
    data_dir =  folders[m]
    print(data_dir)
    stats_file = [filename for filename in os.listdir(data_dir) if filename.startswith("stats_df")]

    sep_list = list(set([stats_file[i].split('stats_df_sep')[1].split('_proc')[0] for i in range(len(stats_file))]))
    sep_numerical_list = [float(a) for a in sep_list]
    sep_list_sorted = [x for _,x in sorted(zip(sep_numerical_list,sep_list))]
    proc_list = list(set([stats_file[i].split('_proc')[1].split('_superportion')[0] for i in range(len(stats_file))]))
    proc_numerical_list = [float(a) for a in proc_list]
    proc_list_sorted = [x for _,x in sorted(zip(proc_numerical_list,proc_list))]
    proc_numerical_list_sorted = [int(a) for a in proc_list_sorted]
    superportion_list = list(set([stats_file[i].split('superportion')[1].split('_superproc')[0] for i in range(len(stats_file))]))
    superportion_numerical_list = [float(a) for a in superportion_list]
    superportion_list_sorted = [x for _,x in sorted(zip(superportion_numerical_list,superportion_list))]
    superportion_numerical_sorted = [float(a) for a in superportion_list_sorted]
    superproc_list = list(set([stats_file[i].split('superproc')[1].split('_ctcf')[0] for i in range(len(stats_file))]))
    superproc_numerical_list = [float(a) for a in superproc_list]
    superproc_list_sorted = [x for _,x in sorted(zip(superproc_numerical_list,superproc_list))]
    superproc_numerical_sorted = [int(a) for a in superproc_list_sorted]
    ctcf_list = list(set([stats_file[i].split('ctcf')[1].split('_dsb')[0] for i in range(len(stats_file))]))
    ctcf_numerical_list = [float(a) for a in ctcf_list]
    ctcf_list_sorted = [x for _,x in sorted(zip(ctcf_numerical_list,ctcf_list))]
    ctcf_numerical_sorted = [float(a) for a in ctcf_list_sorted]
    dsb_list = list(set([stats_file[i].split('dsb')[1].split('_superloading')[0] for i in range(len(stats_file))]))
    dsb_numerical_list = [float(a) for a in dsb_list]
    dsb_list_sorted = [x for _,x in sorted(zip(dsb_numerical_list,dsb_list))]
    superloading_list = list(set([stats_file[i].split('superloading')[1].split('_bs')[0] for i in range(len(stats_file))]))
    superloading_numerical_list = [float(a) for a in superloading_list]
    superloading_list_sorted = [x for _,x in sorted(zip(superloading_numerical_list,superloading_list))]
    superloading_numerical_sorted = [float(a) for a in superloading_list_sorted]
    superloading_ratio = [np.around(100*superloading/(superloading+1e4),decimals=0) for superloading in superloading_numerical_sorted]
    bs_list = list(set([stats_file[i].split('bs')[1].split('.csv')[0] for i in range(len(stats_file))]))
    bs_numerical_list = [float(a) for a in bs_list]
    bs_list_sorted = [x for _,x in sorted(zip(bs_numerical_list,bs_list))]
    bs_numerical_sorted = [float(a) for a in bs_list_sorted]

    colors = ['cyan','dodgerblue','blueviolet','violet','mediumorchid']
    matplotlib.rcParams.update({'font.size': 22})       
    fig, axs = plt.subplots(1,len(bs_list_sorted),figsize=(65,12))

    legend = ['successful repairs','failed repairs']

    if m == 0:
        ctcf_bs_fp = np.zeros((len(folders),len(bs_list_sorted)))
        ctcf_bs_success =  np.zeros((len(folders),len(bs_list_sorted)))
        ctcf_bs_realtime =  np.zeros((len(folders),len(bs_list_sorted)))
        ctcf_bs_time0 =    np.zeros((len(folders),len(bs_list_sorted)))

        
    for i in range(len(proc_list_sorted)):
        for j in range(len(bs_list_sorted)):
            par_combo = 'sep' + sep_list_sorted[0] + '_proc' + proc_list_sorted[i] + '_superportion' + superportion_list_sorted[0] + '_superproc' + str(proc_numerical_list_sorted[i]*20) + '_ctcf' + ctcf_list_sorted[0] + '_dsb' + dsb_list_sorted[0] + '_superloading' + superloading_list_sorted[0]+ '_bs' + bs_list_sorted[j]
            filename = data_dir + '/stats_df_' + par_combo + '.csv'
            if path.exists(filename):
                stats_df = pd.read_csv(filename)
                stats_df = stats_df.drop(columns='Unnamed: 0')
                stats_df_threshold1 = stats_df.loc[stats_df['threshold']==1]
                sep_proc_1st_pass = stats_df_threshold1['first_passage_time'].to_numpy()
                if len(sep_proc_1st_pass)>0:
                    ctcf_bs_fp[m,j]=np.mean(sep_proc_1st_pass)*base_stochasticity*2/60 
                else:
                    ctcf_bs_fp[m,j]=np.nan


                portion_df = pd.read_csv(data_dir + '/restrained_df_' + par_combo + '.csv')
                portion_df = portion_df.drop(columns='Unnamed: 0')
                portion_df_threshold1 = portion_df.loc[portion_df['threshold']==1]
                ctcf_bs_time0[m,j] = portion_df_threshold1['restrained proportion time0'].to_numpy()[0]
                ctcf_bs_realtime[m,j] = portion_df_threshold1['restrained proportion realtime'].to_numpy()[0]
                ctcf_bs_success[m,j] = portion_df_threshold1['repaired proportion'].to_numpy()[0] * 100
    
    
                gapsize_file = data_dir +'/GapLength_'+par_combo+'.npy'
                with open(gapsize_file, 'rb') as infile:
                    constrained_failed_array = np.load(infile)
                    constrained_array = np.load(infile)
                                         
#                 sum_constrained_array=np.vstack((constrained_array,constrained_failed_array))
                axs[j].hist(constrained_array, histtype=u'step',density=True, cumulative=True, linewidth=2, alpha=1,range=[0,np.amax([np.amax(constrained_array),np.amax(constrained_failed_array)])])
                axs[j].hist(constrained_failed_array ,histtype=u'step',density=True,cumulative=True,  linewidth=2,alpha=1,range=[0,np.amax([np.amax(constrained_array),np.amax(constrained_failed_array)])])
                axs[j].set_ylabel('density')
                axs[j].set_xlabel('constraining LEF size(kb)')
                axs[j].set_xlim(0,1600)
                axs[j].set_xticks([0,400,800,1200,1600])
                axs[j].legend(legend,loc='upper left')
                axs[j].tick_params(direction='out', length=12, width=2)
                axs[j].set_title("Boundary strength = "+bs_list_sorted[j])
                if j==(len(bs_list_sorted)-1):
                    plt.savefig('Manuscript/ConstrainingLEFsizeDistribution_v3.pdf',format='pdf',bbox_inches='tight')
                    plt.show()




